# Preparación del entorno: instalaciones y librerías

In [1]:
#Instalaciones
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
#Librerías
import os
import pandas as pd
import glob
import shutil
from google.colab import drive
import torch.nn.functional as F
from torch.optim import Adam

#Semilla
SEED = 42

#Librerías donde fijar semilla para garantizar reproducibilidad
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
import torch
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
import seaborn as sns
import matplotlib.pyplot as plt
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, set_seed
set_seed(SEED)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score, confusion_matrix
from sklearn.model_selection import ParameterGrid

# Dataset

In [3]:
#Carga del dataset de España
dataset_es = load_dataset("alttal/balanced_SA_opiniones_indumentaria_ES_Espanya_splits")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/625 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/313 [00:00<?, ? examples/s]

In [4]:
dataset_es

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3006
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 313
    })
})

# Métricas de evaluación

In [5]:
#Definimos las métricas para evaluar el modelo
#Este conjunto de métricas depende de la tarea.
#Para clasificación de textos se suelen utilizar accuracy, precision, recall y F1-score.

def compute_metrics(pred):
  y_true = pred.label_ids              # son las etiquetas reales
  y_pred = pred.predictions.argmax(-1) # son las predicciones
  acc = accuracy_score(y_true, y_pred)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

  #métricas por clase
  precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(y_true, y_pred, average=None)

  #conversiones necesarias para evitar errores de registro en Tensorboard
  precision_per_class = list(precision_per_class)
  recall_per_class = list(recall_per_class)
  f1_per_class = list(f1_per_class)

  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'precision_per_class': precision_per_class,
      'recall_per_class': recall_per_class,
      'f1_per_class': f1_per_class
  }

# Documentación del progreso

In [7]:
#Montar unidad de disco
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#Ruta raíz para guardar cada experimento
path = '/content/drive/My Drive/TFM/Experimentos balanced'

#Cambiar directorio actual de trabajo
os.chdir(path)

#Verificar directorio actual
print("Directorio actual:", os.getcwd())

Directorio actual: /content/drive/My Drive/TFM/Experimentos balanced


In [9]:
#Definición de una función para eliminar logs antiguos (para evitar falta de almacenamiento)
def clean_old_logs(logging_dir, keep_last_n=1):
    log_files = sorted(glob.glob(os.path.join(logging_dir, '*')), key=os.path.getmtime)
    for log_file in log_files[:-keep_last_n]:
        os.remove(log_file)

In [10]:
#Ruta al directorio de logs
log_dir = path

#Eliminar archivos antiguos de TensorBoard (para evitar acumulación de demasiados logs)
for file in os.listdir(log_dir):
    if file.startswith('events.out.tfevents'):
        os.remove(os.path.join(log_dir, file))

# Modelo 4

##Carga del modelo 4 y adaptación de los datos

In [11]:
#Carga del modelo pre-entrenado
model4 = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis", num_labels=3)

config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

In [12]:
#A modo de información, vemos la configuración del modelo
model4.config

RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.42.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30002
}

In [13]:
#¿Cómo son las etiquetas del modelo? Importante porque deben ser iguales a las del dataset
model4.config.id2label

{0: 'NEG', 1: 'NEU', 2: 'POS'}

##Tokenización del dataset

In [14]:
#Carga del tokenizador específico
tokenizer4 = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
#Se define tokenizador del primer modelo
def tokenize_function4(example):
    return tokenizer4(example["text"], padding='longest', truncation=True)

In [16]:
#Tokenización del dataset
encoded_dataset_es_1 = dataset_es.map(tokenize_function4, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_es_1 = encoded_dataset_es_1.remove_columns(["text"])
encoded_dataset_es_1

Map:   0%|          | 0/3006 [00:00<?, ? examples/s]

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

Map:   0%|          | 0/313 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3006
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 313
    })
})

In [17]:
#Ejemplo de la tokenización en cada split
print(encoded_dataset_es_1['train'][0])
print(encoded_dataset_es_1['validation'][0])
print(encoded_dataset_es_1['test'][0])

{'label': 0, 'input_ids': [0, 560, 474, 3969, 531, 3250, 443, 442, 507, 16230, 474, 3839, 22368, 464, 3839, 8568, 1775, 694, 446, 4539, 609, 413, 706, 601, 2755, 3435, 463, 752, 3013, 464, 497, 1744, 512, 2231, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Experimento 4.1.

In [18]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/model4_lit"
shutil.make_archive("./outputs/model4_lit", 'zip', output_dir)
logging_dir = "./logs/model4_lit"
shutil.make_archive("./logs/model4_lit", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros recomendados

In [19]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [20]:
#Definición de hiperparámetros
training_args_model4_lit = TrainingArguments(
    output_dir=output_dir,           # Directorio de salida
    logging_dir=logging_dir,         # Directorio para los logs
    eval_strategy="steps",           # Estrategia de evaluación
    save_strategy="steps",           # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                 # Número de pasos entre cada guardado
    logging_strategy="steps",        # Estrategia de registro
    logging_steps=1000,              # Número de pasos entre cada registro
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch de entrenamiento (podría ser también 32)
    per_device_eval_batch_size=16,   # Tamaño del batch de evaluación (podría ser también 32)
    learning_rate=3e-5,              # Tasa de aprendizaje
    weight_decay=0.01,               # Decadencia de peso (podría ser también 0.1)
    warmup_ratio=0.1,                # Proporción de épocas de calentamiento
    save_total_limit=1,              # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=SEED,                       # Semilla para garantizar reproducibilidad
    fp16=True,                       # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",      # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [21]:
#Definición de optimizador
optimizer = Adam(model4.parameters(), lr=3e-5, weight_decay=0.01)

In [22]:
#Definición de un objeto de la clase Trainer para el primer experimento con el modelo 1
trainer_model4_lit = Trainer(
    model = model4,
    args = training_args_model4_lit,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer4,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer4),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Entrenamiento

In [23]:
trainer_model4_lit.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=564, training_loss=0.537250816399324, metrics={'train_runtime': 40.4822, 'train_samples_per_second': 222.765, 'train_steps_per_second': 13.932, 'total_flos': 593189200267776.0, 'train_loss': 0.537250816399324, 'epoch': 3.0})

Validación

In [24]:
trainer_model4_lit.evaluate()

Trainer is attempting to log a value of "[0.8869565217391304, 0.4857142857142857, 0.9453125]" of type <class 'list'> for key "eval/precision_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.816, 0.7555555555555555, 0.8461538461538461]" of type <class 'list'> for key "eval/recall_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.85, 0.5913043478260869, 0.8929889298892989]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.4780034124851227,
 'eval_accuracy': 0.8210862619808307,
 'eval_f1': 0.7780977592384618,
 'eval_precision': 0.7726611024844722,
 'eval_recall': 0.8059031339031338,
 'eval_precision_per_class': [0.8869565217391304,
  0.4857142857142857,
  0.9453125],
 'eval_recall_per_class': [0.816, 0.7555555555555555, 0.8461538461538461],
 'eval_f1_per_class': [0.85, 0.5913043478260869, 0.8929889298892989],
 'eval_runtime': 0.4476,
 'eval_samples_per_second': 699.279,
 'eval_steps_per_second': 44.682,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [25]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model4_lit.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.89      0.82      0.85       125
           1       0.49      0.76      0.59        45
           2       0.95      0.85      0.89       143

    accuracy                           0.82       313
   macro avg       0.77      0.81      0.78       313
weighted avg       0.86      0.82      0.83       313



In [26]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [27]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model4_lit.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.92      0.83      0.88       248
           1       0.45      0.72      0.55        90
           2       0.93      0.83      0.87       287

    accuracy                           0.81       625
   macro avg       0.77      0.79      0.77       625
weighted avg       0.86      0.81      0.83       625



In [28]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Grid Search

Búsqueda de hiperparámetros optimizados con la técnica Grid Search:

In [29]:
#Definición del espacio de búsqueda
param_grid = {
    'num_train_epochs': [3, 5],
    'per_device_train_batch_size': [16, 32],
    'per_device_eval_batch_size': [16, 32],
    'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5, 1e-4],
    'weight_decay': [0.01, 0.1]
}

grid = ParameterGrid(param_grid)

In [30]:
#Función para calcular el f1-score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        'f1': f1_score(labels, predictions, average='macro')
    }

In [31]:
#Inicialización de variables para guardar los mejores resultados
best_score = 0
best_params = None

In [32]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/model4_search"
shutil.make_archive("./outputs/model4_search", 'zip', output_dir)
logging_dir = "./logs/model4_search"
shutil.make_archive("./logs/model4_search", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

In [33]:
#Bucle para probar los diferentes parámetros
for params in grid:
    print(f"Entrenando con parámetros: {params}")

    #Definición de hiperparámetros
    training_args_model4_search = TrainingArguments(
        output_dir=output_dir,                                                  # Directorio de salida
        logging_dir=logging_dir,                                                # Directorio para los logs
        eval_strategy="steps",                                                  # Estrategia de evaluación
        save_strategy="steps",                                                  # Estrategia de guardado (debe coincidir con la de evaluación)
        save_steps=1000,                                                        # Número de pasos entre cada guardado
        logging_strategy="steps",                                               # Estrategia de registro
        logging_steps=1000,                                                     # Número de pasos entre cada registro
        num_train_epochs=params['num_train_epochs'],                            # Número de épocas de entrenamiento
        per_device_train_batch_size=params['per_device_train_batch_size'],      # Tamaño del batch de entrenamiento
        per_device_eval_batch_size=params['per_device_eval_batch_size'],        # Tamaño del batch de evaluación
        learning_rate=params['learning_rate'],                                  # Tasa de aprendizaje
        weight_decay=params['weight_decay'],                                    # Decadencia de peso
        save_total_limit=1,                                                     # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
        seed=SEED,                                                              # Semilla para garantizar reproducibilidad
        fp16=True,                                                              # Uso de fp16 para acelerar el entrenamiento
        load_best_model_at_end=True,                                            # Cargar el mejor modelo al final del entrenamiento
        metric_for_best_model='f1',                                             # Métrica objetivo, a optimizar para encontrar el mejor modelo
    )

    #Definición de un objeto de la clase Trainer para la búsqueda con el modelo 4
    trainer_model4_search = Trainer(
        model=model4,
        args=training_args_model4_search,
        train_dataset=encoded_dataset_es_1['train'],
        eval_dataset=encoded_dataset_es_1['validation'],
        tokenizer=tokenizer4,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer4),
        compute_metrics=compute_metrics,
        optimizers = (optimizer, None),
        callbacks = [early_stopping]
    )

    #Entrenamiento
    trainer_model4_search.train()

    #Limpiar logs antiguos después del entrenamiento en cada iteración
    clean_old_logs(logging_dir, keep_last_n=1)

    #Validación
    eval_result = trainer_model4_search.evaluate()

    #Guardamos métrica objetivo
    score = eval_result.get('eval_f1', 0)

    #Siempre que la métrica objetivo sea mejor que el score guardado hasta el momento, se lo actualiza
    if score > best_score:
        best_score = score
        best_params = params

#Vemos los resultados del Grid Search
print(f"Mejores parámetros: {best_params}")
print(f"Mejor puntuación F1: {best_score}")

Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.01}


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.1}


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


...

Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 5, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'weight_decay': 0.1}


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss


Mejores parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 32, 'weight_decay': 0.1}
Mejor puntuación F1: 0.7447405340464287


## Experimento 4.2.

In [34]:
#Documentación
#Definición de directorios para el segundo experimento
output_dir = "./outputs/model4_opt"
shutil.make_archive("./outputs/model4_opt", 'zip', output_dir)
logging_dir = "./logs/model4_opt"
shutil.make_archive("./logs/model4_opt", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros optimizados

In [35]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [36]:
#Definición de hiperparámetros (basados en Grid Search)
training_args_model4_opt = TrainingArguments(
    output_dir=output_dir,                                # Directorio de salida
    logging_dir=logging_dir,                              # Directorio para los logs
    eval_strategy="steps",                                # Estrategia de evaluación
    save_strategy="steps",                                # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                                      # Número de pasos entre cada guardado
    logging_strategy="steps",                             # Estrategia de registro
    logging_steps=1000,                                   # Número de pasos entre cada registro
    num_train_epochs=3,                                   # Número de épocas de entrenamiento
    per_device_train_batch_size=32,                       # Tamaño del batch de entrenamiento
    per_device_eval_batch_size=16,                        # Tamaño del batch de evaluación
    learning_rate=1e-05,                                  # Tasa de aprendizaje
    weight_decay=0.1,                                     # Decadencia de peso
    save_total_limit=1,                                   # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=42,                                              # Semilla para garantizar reproducibilidad
    fp16=True,                                            # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,                          # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",                           # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [37]:
#Definición de optimizador
optimizer = Adam(model4.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])

In [38]:
#Definición de un objeto de la clase Trainer para el segundo experimento con el modelo 1
trainer_model4_opt = Trainer(
    model = model4,
    args = training_args_model4_opt,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer4,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer4),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Entrenamiento

In [39]:
trainer_model4_opt.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=282, training_loss=0.9675209640611148, metrics={'train_runtime': 22.3291, 'train_samples_per_second': 403.868, 'train_steps_per_second': 12.629, 'total_flos': 593189200267776.0, 'train_loss': 0.9675209640611148, 'epoch': 3.0})

Validación

In [40]:
trainer_model4_opt.evaluate()

{'eval_loss': 1.0620288848876953,
 'eval_f1': 0.6265631618032026,
 'eval_runtime': 0.4691,
 'eval_samples_per_second': 667.191,
 'eval_steps_per_second': 42.632,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [41]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model4_opt.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.77      0.72      0.74       125
           1       0.28      0.44      0.34        45
           2       0.85      0.74      0.79       143

    accuracy                           0.69       313
   macro avg       0.63      0.64      0.63       313
weighted avg       0.74      0.69      0.71       313



In [42]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [43]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model4_opt.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       248
           1       0.29      0.49      0.36        90
           2       0.80      0.66      0.72       287

    accuracy                           0.65       625
   macro avg       0.61      0.62      0.60       625
weighted avg       0.70      0.65      0.67       625



In [44]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento extra 4.3.

In [45]:
#Documentación
#Definición de directorios para el tercer experimento
output_dir = "./outputs/model4_est"
shutil.make_archive("./outputs/model4_est", 'zip', output_dir)
logging_dir = "./logs/model4_est"
shutil.make_archive("./logs/model4_est", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros originales

In [46]:
#Definición de hiperparámetros estándar
training_args_model4_estandar = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    seed=42,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [47]:
#Definición de un objeto de la clase Trainer para el tercer experimento con el modelo 1
trainer_model4_estandar = Trainer(
    model = model4,
    args = training_args_model4_estandar,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer4,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer4)
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Entrenamiento

In [48]:
trainer_model4_estandar.train()

Step,Training Loss,Validation Loss,F1
1000,0.783200,0.914527,0.572885


TrainOutput(global_step=1128, training_loss=0.7585512458855379, metrics={'train_runtime': 77.8584, 'train_samples_per_second': 115.826, 'train_steps_per_second': 14.488, 'total_flos': 593189200267776.0, 'train_loss': 0.7585512458855379, 'epoch': 3.0})

Validación

In [49]:
trainer_model4_estandar.evaluate()

{'eval_loss': 0.9145273566246033,
 'eval_f1': 0.5728852017769444,
 'eval_runtime': 0.8284,
 'eval_samples_per_second': 377.822,
 'eval_steps_per_second': 48.284,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [50]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model4_estandar.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.80      0.57      0.66       125
           1       0.22      0.33      0.26        45
           2       0.76      0.83      0.79       143

    accuracy                           0.65       313
   macro avg       0.59      0.58      0.57       313
weighted avg       0.70      0.65      0.66       313



In [51]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [52]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model4_estandar.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.79      0.64      0.71       248
           1       0.29      0.40      0.34        90
           2       0.75      0.79      0.77       287

    accuracy                           0.67       625
   macro avg       0.61      0.61      0.60       625
weighted avg       0.70      0.67      0.68       625



In [53]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento final: mejor modelo, nuevo dataset

In [54]:
#Elección del mejor modelo
#Modelo final -> Modelo con hiperparámetros recomendados
#Carga del nuevo dataset: comentarios de Argentina
dataset_ar = load_dataset("alttal/SA_opiniones_indumentaria_ES_Argentina_splits")

Generating train split:   0%|          | 0/2187 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/625 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/313 [00:00<?, ? examples/s]

In [55]:
#Unimos los splits: sólo necesitaremos uno para el test
from datasets import concatenate_datasets
dataset_ar = concatenate_datasets([dataset_ar['train'], dataset_ar['test'], dataset_ar['validation']])

#Ejemplo
print(dataset_ar)

Dataset({
    features: ['id', 'label', 'text', '__index_level_0__'],
    num_rows: 3125
})


In [56]:
#Se eliminan columnas innecesarias
dataset_ar = dataset_ar.remove_columns(["id","__index_level_0__"])
dataset_ar

Dataset({
    features: ['label', 'text'],
    num_rows: 3125
})

In [57]:
#Las etiquetas de los datasets son strings. Se las cambia a valores numéricos
def label_to_int(label):
    if label == "POS":
        return 2
    elif label == "NEG":
        return 0
    else:
        return 1

dataset_ar_1 = dataset_ar.map(lambda x: {"label": label_to_int(x["label"])})

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

In [58]:
#Tokenización del dataset
encoded_dataset_ar_1 = dataset_ar_1.map(tokenize_function4, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_ar_1 = encoded_dataset_ar_1.remove_columns(["text"])
encoded_dataset_ar_1

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3125
})

Reporte de clasificación sobre el conjunto de datos de Argentina

In [59]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model4_lit.predict(encoded_dataset_ar_1)
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.58      0.54      0.56       703
           1       0.23      0.31      0.26       333
           2       0.86      0.83      0.85      2089

    accuracy                           0.71      3125
   macro avg       0.56      0.56      0.56      3125
weighted avg       0.73      0.71      0.72      3125

